In [ ]:
# Setup Spark SQL
# Note if running locally you need the JVM https://www.oracle.com/java/technologies/downloads/ 
# Consider running in https://colab.research.google.com/
%pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder.appName("SQLIntro").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('WARN')


In [ ]:
# Download Guteburg Book of Mormon and Bible
!curl -L https://ia601205.us.archive.org/18/items/thebookofmormon00017gut/mormon13.txt > bookOfMormon.txt
!curl -L https://www.gutenberg.org/cache/epub/10/pg10.txt > bible.txt

In [ ]:
def putBookIntoWordsTable(bookFileTxt, tableName):
    import re
    lines = sc.textFile(bookFileTxt)
    linesLower = lines.map(lambda line: line.lower())
    words = linesLower.flatMap(lambda lowerLine: re.split('[^a-z]', lowerLine))
    asRows = words.map(lambda w: Row(word=w))
    asDF = asRows.toDF()
    asDF.createOrReplaceTempView(tableName)

putBookIntoWordsTable("bookOfMormon.txt", "bookOfMormon")
putBookIntoWordsTable("bible.txt", "bible")

In [ ]:
bookOfMormonOnlyWords = spark.sql("""
                
                with uniqueToBookOfMormon as (
                    select * from bookOfMormon except select * from bible
                )
                select word, count(*) as counts from 
                bookOfMormon
                where word in (select word from uniqueToBookOfMormon)
                group by word
                order by counts desc
                
                """)

bookOfMormonOnlyWords.rdd.map(lambda row: (row.word, row.counts)).take(100)


In [ ]:
bibleOnlyWords = spark.sql("""
                
                with uniqueToBible as (
                    select * from bible except select * from bookOfMormon
                )
                select word, count(*) as counts from 
                bible
                where word in (select word from uniqueToBible)
                group by word
                order by counts desc
                
                """)

bibleOnlyWords.rdd.map(lambda row: (row.word, row.counts)).take(100)
